In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std


print('Question 1')
print('Q1: The Civilian Complaint Review Board (CCRB) of New York City releases anonymized data about complaints it has received reguarding civilian interactions with police. Besides their aggregated reporting data, they release record-level data as an Excel spreadsheet. Download this data. A description of the data appears in the first sheet of the spreadsheet, which could be useful to your analysis.')

Question 1
Q1: The Civilian Complaint Review Board (CCRB) of New York City releases anonymized data about complaints it has received reguarding civilian interactions with police. Besides their aggregated reporting data, they release record-level data as an Excel spreadsheet. Download this data. A description of the data appears in the first sheet of the spreadsheet, which could be useful to your analysis.


In [44]:
print("1. How many unique complaints (identified by 'UniqueComplaintId') with complete information (i.e. there are no missing values) appear in the dataset?")

1. How many unique complaints (identified by 'UniqueComplaintId') with complete information (i.e. there are no missing values) appear in the dataset?


In [45]:
data = pd.read_excel('ccrb_datatransparencyinitiative_20170207.xlsx', sheet_name='Complaints_Allegations')

In [46]:
df = pd.DataFrame(data, columns=data.columns)
df.dropna(inplace=True)
ndf = df.drop_duplicates('UniqueComplaintId')
len(ndf['UniqueComplaintId'].unique().tolist())

68467

In [ ]:
print('2. What proportion of complaints occur in the borough with the largest number of complaints? For this question, only consider unique complaints with complete information. ')

In [ ]:
sns.countplot(ndf['Borough of Occurrence'])
plt.title('Borough of Occurrence')

In [ ]:
bindex = ndf['Borough of Occurrence'].value_counts().index.tolist()
bvalues = ndf['Borough of Occurrence'].value_counts().values.tolist()
[x+': '+ str(y) for x,y in zip(bindex,bvalues)]

In [ ]:
ans2 = max(bvalues) / sum(bvalues)

In [ ]:
print("Answer = " + str(ans2))

In [ ]:
print('3. How many complaints per 100k residents were there in the borough with the highest number of complaints per capita resulting from incidents in 2016? Find the 2016 population estimates of each borough on Wikipedia. Ignore complaints from "Outside NYC". For this question, only consider unique complaints with complete information. ')

In [ ]:
df = ndf.copy()
#trend.set_index('Incident Year', inplace=True)
df = df[df['Incident Year'].isin([int(2016)])]
local = bindex[0:5]
df = df[df['Borough of Occurrence'].isin(local)]

In [ ]:
sns.countplot(df['Borough of Occurrence'])
#complainers = df[df['Borough of Occurrence'].isin(['Brooklyn'])]

In [ ]:
a = df['Borough of Occurrence'].value_counts().index.tolist()
b = df['Borough of Occurrence'].value_counts().values.tolist()
local_2016 = {a: b for a,b in zip(a,b)}
local_2016

In [ ]:
pop_2016 = {'Brooklyn': 2629150, 'Manhattan': 1643734, 
            'Bronx': 1455720, 'Queens': 2333054, 'Staten Island': 476015}

percapita = {k: local_2016[k]/pop_2016[k] for k in local_2016}
val = max(percapita, key=percapita.get)
print(val)

In [ ]:
per100k = percapita['Bronx'] * 100000
print(str(per100k) + ' complaints per 100k residents')

In [ ]:
print('Q4.')

In [ ]:
cdt = ndf.copy()
cdt.head()

In [ ]:
cdt['Availability'] = cdt['Close Year'] - cdt['Received Year']

In [ ]:
avg_duration = cdt['Availability'].mean()
print(str(avg_duration) + ' years' )

In [ ]:
print('Q5. Linear Regression')
fsk = ndf.copy()
fsk = fsk[fsk['Complaint Contains Stop & Frisk Allegations'].isin([True])]
time = np.arange(2007,2017)
fsk = fsk[fsk['Incident Year'].isin(time)]
sns.catplot(x="Incident Year", kind="count", palette="ch:.1", height=1.5, aspect=4, data=fsk)

In [ ]:
a = fsk['Incident Year'].value_counts().index.tolist()
b = fsk['Incident Year'].value_counts().values.tolist()
f = {str(a): b for a,b in zip(a,b)}
q = pd.DataFrame.from_dict(f, orient='index', columns=['count'])
f.items()


In [ ]:
wing = ndf.copy()

In [ ]:
type(q.index)
df = q.reset_index()
type(df['index'])
df = df.rename(index=str, columns={"index": "date", "count": "num_incidents"})
df.index = pd.DatetimeIndex(df.date)
idx = pd.date_range(df.index.min(), df.index.max())

In [ ]:
df = df.reindex(index = idx)
df.dropna(inplace=True)
tempdf = df.copy()
df

In [ ]:
df.insert(df.shape[1],
          'row_count',
          df.index.value_counts().sort_index().cumsum())

In [ ]:
pd.to_datetime(df['date']).astype(int).astype(float)
results = ols('row_count ~ num_incidents', data=df).fit()
print(results.summary())

In [ ]:
res = sm.OLS(df.num_incidents, df.row_count).fit()
print('Predicted values: ', res.predict())

In [ ]:
fig = sns.regplot(data = df, x = 'row_count', y = 'num_incidents')
plt.show()
df

In [ ]:
fig = sns.lmplot(data = df, x = 'row_count', y = 'num_incidents')

In [ ]:
dta.index = pd.Index(sm.tsa.datetools.dates_from_range('2007', '2016'))
del dta["date"]

In [ ]:
dta
dta.plot(figsize=(12,4))

In [ ]:
df

In [ ]:
endog = pd.Series(dta.num_incidents, index=dates)
endog

In [ ]:
ar_model = sm.tsa.AR(endog, freq='A')

In [ ]:
pandas_ar_res = ar_model.fit(maxlag=8, method='mle', disp=-1)

In [ ]:
pred = pandas_ar_res.predict(start='2007', end='2018')
print(pred.round())

Answer = 387

In [ ]:
print('Q6. Calculate the chi-square test statistic for testing whether a complaint is more likely to receive a full investigation when it has video evidence. For this question, only consider unique complaints with complete information.')

In [ ]:
full = wing.copy()

In [ ]:
#full = wing[wing['Is Full Investigation'].isin([True])]
full.loc[full['Complaint Has Video Evidence']==False, 'Video_Dummy'] = 0
full.loc[full['Complaint Has Video Evidence']==True, 'Video_Dummy'] = 1
full.loc[full['Is Full Investigation']==False, 'Partial_Investigation'] = 1
full.loc[full['Is Full Investigation']==True, 'Partial_Investigation'] = 0
full.loc[full['Is Full Investigation']==True, 'Full_Investigation'] = 1
full.loc[full['Is Full Investigation']==False, 'Full_Investigation'] = 0

In [ ]:
a = full['Is Full Investigation'].value_counts().index.tolist()
b = full['Is Full Investigation'].value_counts().values.tolist()
f = {str(a): b for a,b in zip(a,b)}
p = pd.DataFrame.from_dict(f, orient='index', columns=['count'])
full

In [173]:
evi = ols('Video_Dummy ~ Full_Investigation', data=df).fit()

PatsyError: Error evaluating factor: NameError: name 'Full_Investigation' is not defined
    Video_Dummy ~ Full_Investigation
                  ^^^^^^^^^^^^^^^^^^

In [ ]:
df.loc[df['type']=='Run', 'average_watts'] = 0